In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import Subset
from torchvision import datasets, transforms

In [3]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.__seq = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def forward(self, input):
        return self.__seq(input)

In [4]:
class Loader:
    def __init__(self, train_loader, valid_loader, test_loader):
        self.train_loader = train_loader
        self.valid_loader = valid_loader
        self.test_loader = test_loader


class ClassDist:
    def __init__(self):
        self.train_dist = dict()
        self.valid_dist = dict()
        self.test_dist = dict()


class ModelResult:
    def __init__(self):
        self.accuracy = 0.
        self.class_accuracy = dict()
        self.class_dist = ClassDist()


class TrainResult(ModelResult):
    def __init__(self):
        super().__init__()
        self.train_loss = []
        self.valid_loss = []


class TestResult(ModelResult):
    def __init__(self):
        super().__init__()
        self.test_loss = []

In [5]:
cifar10_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [6]:
class BaseNet(nn.Module):
    def __init__(self, labels, device=torch.device('cpu')) -> None:
        super().__init__()
        self._labels = labels
        self._device = device

    def forward(self, img):
        return None

    def to_device(self):
        return self.to(self._device)

    def base_train_model(self, train_loader, valid_loader, loss_fn, optim, eporchs, filename=None, verbose=True):
        result = TrainResult()
        for label in self._labels:
            result.class_accuracy[label] = 0.
        prev_loss = torch.inf
        train_samples_per_class = [0 for i in range(len(self._labels))]
        valid_correct_per_class = [0 for i in range(len(self._labels))]
        valid_samples_per_class = [0 for i in range(len(self._labels))]
        # Loop through each eporch
        for i in range(eporchs):
            self.train()
            curr_loss = 0.
            # Train the model
            for images, labels in train_loader:
                images = images.to(self._device)
                labels = labels.to(self._device)
                # Reset the gradient
                self.zero_grad()
                # Predict image labels
                output = self.forward(images)
                pred = torch.argmax(output, dim=1)
                m_labels = labels.view_as(pred)
                for j in range(len(m_labels)):
                    train_samples_per_class[m_labels[j]] += 1
                # Compute the loss, add it to the total loss, and backpropagates
                loss = loss_fn(output, labels)
                curr_loss += loss.item()
                loss.backward()
                # Run the optimizer
                optim.step()
            result.train_loss.append(curr_loss)
            self.eval()
            curr_loss = 0.
            accuracy = 0
            # Validate the model
            with torch.no_grad():
                for images, labels in valid_loader:
                    images = images.to(self._device)
                    labels = labels.to(self._device)
                    # Predict image labels
                    output = self.forward(images)
                    pred = torch.argmax(output, dim=1)
                    m_labels = labels.view_as(pred)
                    accuracy += pred.eq(m_labels).sum()
                    for j in range(len(m_labels)):
                        valid_samples_per_class[m_labels[j]] += 1
                        if pred[j] == m_labels[j]:
                            valid_correct_per_class[m_labels[j]] += 1
                    # Compute the loss, add it to the total loss
                    loss = loss_fn(output, labels)
                    curr_loss += loss.item()
            result.valid_loss.append(curr_loss)
            accuracy = accuracy / len(valid_loader.dataset) * 100
            # Aggregate the accuracy
            result.accuracy += accuracy
            if verbose:
                print()
                print(f"Eporch {i + 1} / {eporchs}: training loss = {result.train_loss[-1]: .3f},\
                        validation loss = {result.valid_loss[-1]: .3f},\
                        accuracy = {accuracy: .3f}%")
            # If the current total loss is smaller than the previous total loss, save the updated model to the file
            if filename and curr_loss < prev_loss:
                if verbose:
                    print('Saving model as the validation loss decreases...')
                torch.save(self.state_dict(), filename)
                prev_loss = curr_loss
        # Compute the mean accuracy
        result.accuracy /= eporchs
        for k in range(len(self._labels)):
            # Compute the accuracy per class
            result.class_accuracy[self._labels[k]] = round(
                valid_correct_per_class[k] / valid_samples_per_class[k] * 100, 3)
            # Get the distribution per class for the training set
            result.class_dist.train_dist[self._labels[k]] = round(
                (train_samples_per_class[k] / len(train_loader.dataset) * 100 / eporchs), 3)
            # Get the distribution per class for the validation set
            result.class_dist.valid_dist[self._labels[k]] = round(
                (valid_samples_per_class[k] / len(valid_loader.dataset) * 100 / eporchs), 3)
        return result

    def base_test_model(self, test_loader, loss_fn, verbose=True):
        result = TestResult()
        for label in self._labels:
            result.class_accuracy[label] = 0.
        self.eval()
        curr_loss = 0.
        accuracy = 0
        test_correct_per_class = [0 for i in range(len(self._labels))]
        test_samples_per_class = [0 for i in range(len(self._labels))]
        # Test the model
        with torch.no_grad():
            for images, labels in test_loader:
                images = images.to(self._device)
                labels = labels.to(self._device)
                # Predict image labels
                output = self.forward(images)
                pred = torch.argmax(output, dim=1)
                m_labels = labels.view_as(pred)
                accuracy += pred.eq(m_labels).sum()
                for j in range(len(m_labels)):
                    test_samples_per_class[m_labels[j]] += 1
                    if pred[j] == m_labels[j]:
                        test_correct_per_class[m_labels[j]] += 1
                # Compute the loss, add it to the total loss, and backpropagates
                loss = loss_fn(output, labels)
                curr_loss += loss.item()
        result.test_loss.append(curr_loss)
        accuracy = accuracy / len(test_loader.dataset) * 100
        result.accuracy = accuracy
        if verbose:
            print(f'test loss = {result.test_loss: .3f}, accuracy = {result.accuracy: .3f}%')
        for k in range(len(self._labels)):
            # Compute the accuracy per class
            result.class_accuracy[self._labels[k]] = round(
                test_correct_per_class[k] / test_samples_per_class[k] * 100, 3)
            # Get the distribution per class for the test set
            result.class_dist.test_dist[self._labels[k]] = round(
                test_samples_per_class[k] / len(test_loader.dataset) * 100, 3)
        return result

In [7]:
class ConvNetCifar10(BaseNet):
    def __init__(self, device=torch.device('cpu')):
        super().__init__(cifar10_labels, device)
        # In-channels: the depth of the input, for colored images, it is 3.
        # Out-channels: the number of filtered images (or the number of filters applied to the input,
        # or the depth of the convolutional layer).
        # 32 x 32 x 64
        self.__conv1 = ConvBlock(3, 64)
        # 32 x 32 x 128
        self.__conv2 = ConvBlock(64, 128)
        # 16 x 16 x 128
        self.__pool1 = nn.MaxPool2d(kernel_size=2)
        self.__res1 = nn.Sequential(ConvBlock(128, 128), ConvBlock(128, 128))
        # 16 x 16 x 256
        self.__conv3 = ConvBlock(128, 256)
        # 8 x 8 x 256
        self.__pool2 = nn.MaxPool2d(kernel_size=2)
        # 8 x 8 x 512
        self.__conv4 = ConvBlock(256, 512)
        # 4 x 4 x 512
        self.__pool3 = nn.MaxPool2d(kernel_size=2)
        self.__res2 = nn.Sequential(ConvBlock(512, 512), ConvBlock(512, 512))
        # 1 x 1 x 512
        self.__pool4 = nn.MaxPool2d(kernel_size=4)
        self.__fc = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(512, 10)
        )

    def forward(self, img):
        output = self.__conv1(img)
        output = self.__conv2(output)
        output = self.__pool1(output)
        output = self.__res1(output) + output
        output = self.__conv3(output)
        output = self.__pool2(output)
        output = self.__conv4(output)
        output = self.__pool3(output)
        output = self.__res2(output) + output
        output = self.__pool4(output)
        output = torch.flatten(output, start_dim=1)
        output = self.__fc(output)
        return output

    def train_model(self, train_loader, valid_loader, eporchs, filename=None):
        loss_fn = nn.CrossEntropyLoss()
        optim = torch.optim.Adam(self.parameters(), lr=0.001)
        return super().base_train_model(train_loader, valid_loader, loss_fn, optim, eporchs, filename, verbose=True)

    def test_model(self, test_loader):
        loss_fn = nn.CrossEntropyLoss()
        return super().base_test_model(test_loader, loss_fn, verbose=True)

    def init_loader(self, batch_size):
        # Prepare data
        train_transform = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(45),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261])
        ])
        valid_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261])
        ])
        test_transform = valid_transform
        # Download the data
        big_train_set = datasets.CIFAR10('dataset', train=True, transform=train_transform, download=True)
        big_valid_set = datasets.CIFAR10('dataset', train=True, transform=valid_transform, download=True)
        test_set = datasets.CIFAR10('dataset', train=False, transform=test_transform, download=True)
        # Splitting the data
        num_indices = len(big_train_set)
        indices = list(range(num_indices))
        train_indices, valid_indices = train_test_split(indices)
        train_set = Subset(big_train_set, train_indices)
        valid_set = Subset(big_valid_set, valid_indices)
        # Create dataloaders
        train_loader = DataLoader(train_set, batch_size, shuffle=True)
        valid_loader = DataLoader(valid_set, batch_size, shuffle=True)
        test_loader = DataLoader(test_set, batch_size, shuffle=True)
        return Loader(train_loader, valid_loader, test_loader)

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
eporchs = 30
convnetCifar10 = ConvNetCifar10(device).to_device()
loader = convnetCifar10.init_loader(batch_size=256)
result = convnetCifar10.train_model(loader.train_loader, loader.test_loader, eporchs,
                                    filename='convnet_cifar10.pt')
print(f'validation mean accuracy: {result.accuracy: .3f}%')
print(f'accuracy per class: {result.class_accuracy}%')
print(f'train distribution per class: {result.class_dist.train_dist}')
print(f'valid distribution per class: {result.class_dist.valid_dist}')
eporch_axis = np.arange(0, eporchs, 1)
fig, axis = plt.subplots()
train_line, = axis.plot(eporch_axis, result.train_loss, 'r', label='train')
valid_line, = axis.plot(eporch_axis, result.valid_loss, 'b', label='valid')
axis.legend(handles=[train_line, valid_line])